# Laboratory 1: Property Graphs
### Luis Alfredo Leon Villapún
### Liliia Aliakberova

# Part B Querying
* * *
In this section we will perform the suggested queries

## Creating the connector
As in previous parts, let's first create the connector to handle the messages with Neo4j.

In [3]:
# Uncomment to install extension
#!pip install ipython-cypher

In [4]:
%load_ext cypher

In [7]:
from connector import Neo4jConnector
from getpass import getpass

uri = "neo4j://localhost:7687"
user = "neo4j"
password = getpass("Input your password to connect")
conn = Neo4jConnector(uri, user, password)

Input your password to connect········


## Queries
1. Find the top 3 most cited papers of each conference.

In [8]:
# From documentation site: https://neo4j.com/docs/python-manual/current/

def query_1(tx):
    result = tx.run(
        """
            MATCH (p:Paper)<-[c:CITED_BY]-(a:Paper)-[r:PRESENTED_AT]->(e:Event)
            WITH e, a, count(c) as citations
            ORDER BY citations DESC
            WITH e,a,citations, collect([citations]) as cited
            RETURN e.ConferenceName as conference, a.Title[0..2] as paper, cited[0..2] as cited
        """
    )
    records = list(result)
    summary = result.consume()
    return records, summary

with conn.driver.session(database="neo4j") as session:
    records, summary = session.execute_read(query_1)
    print(records, summary)

[<Record conference='AIAA/CEAS Aeroacoustics Conference' paper=['Beamforming-based wavepacket model for noise environment predictions of tactical aircraft'] cited=[[10]]>, <Record conference='ASEE Annual Conference and Exposition' paper=['Designing a sustainable large-scale project-based learning (PBL) experience for juniors in electrical and computer engineering'] cited=[[10]]>, <Record conference='ASME Design Engineering Technical Conference' paper=['Realizing origami mechanisms from metal sheets'] cited=[[10]]>, <Record conference='ASME Design Engineering Technical Conference' paper=['Fabrication and testing of a MEMS system for injection of DNA into plant cells'] cited=[[10]]>, <Record conference='ASME Design Engineering Technical Conference' paper=['Patterned carbon nanotube growth on stainless steel'] cited=[[10]]>, <Record conference='Fall Technical Meeting of the Western States Section of the Combustion Institute, WSSCI' paper=['Modeling a pressurized coal feed system'] cited=[